## Processing raw EEG data

+ Prepare EEG 
1. Drop unused channels
2. Subtract reference (mastoids)
3. Detrend 
4. Filter
5. Remove bad channels

+ Segment EEG into standard and deviant epochs (ERPs)
1. subtract baseline
2. Reject artefacts
3. Average to get evoked (for each marker/subject/channel separately)

+ Calculate Mismatch response 
1. deviant - standard for a single subject, for example GiepST_D - GiepST_S
2. check differences between channels and subjects

#### Import Packages 

In [1]:
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autoreject import AutoReject # https://autoreject.github.io/stable/index.html
from multiprocessing import Pool

import PATH

from functions import dataset_loading
from functions import epodium

#### Process ePodium dataset

In [7]:
def process_file(file):    
    path = os.path.join(PATH.ePod_dataset, file)
    filename, extension = os.path.splitext(file)  
    
    if extension =='.bdf':
        path_processed = os.path.join(PATH.ePod_processed_autoreject, filename + ".npy")
        path_events = os.path.join(PATH.ePod_processed_autoreject, 'events', filename + ".txt")
        
        if os.path.exists(path_processed) and os.path.exists(path_events):
            print(f"File {file} already processed \n", end = '')
            return
            
        print(f"Processing file: {file}  \n" , end = '')        
        raw = mne.io.read_raw_bdf(path, preload = True, verbose = False)
        events = mne.find_events(raw, verbose = False)
        events_12 = epodium.group_events_12(events)
        
        # Set electrodes
        raw.pick_channels(epodium.channel_names)
        montage = mne.channels.make_standard_montage('standard_1020') 
        raw.info.set_montage(montage, on_missing = 'ignore')
        
        # High-pass filter for detrending
        raw.filter(0.1, None, verbose = False)
        # Create epochs from raw
        try:
            epochs = mne.Epochs(raw, events_12, epodium.event_dictionary, -0.2, 0.8, preload = True, verbose = False)
        except:
            print(f"Not all events in file {file} \n", end = '')
            return
        # Low pass filter for high-frequency artifacts
        epochs.filter(None, 40, verbose = False)
        
        # Reject bad trials and repair bad sensors in EEG
        ar = AutoReject()
        epochs_clean = ar.fit_transform(epochs)  
        
        # Save data and events
        np.save(path_processed, epochs_clean.get_data())        
        np.savetxt(path_events, epochs_clean.events, fmt='%i')

# Multiprocessing
pool = Pool(processes = 4)
pool.map(process_file, os.listdir(PATH.ePod_dataset))


File 116b.bdf already processed 
File 110b.bdf already processed 
File 105a.bdf already processed 
File 124a.bdf already processed 
File 112a.bdf already processed 
File 118b.bdf already processed 
File 105b.bdf already processed 
File 112b.bdf already processed 
File 107a.bdf already processed 
File 106a.bdf already processed 
File 106b.bdf already processed 
Processing file: 107b (deel 1+2).bdf  
File 111a.bdf already processed 
Processing file: 113a.bdf  
File 117b.bdf already processed 
Processing file: 121b(2).bdf  
File 125b.bdf already processed 
File 127a.bdf already processed 
File 125a.bdf already processed 
File 120b.bdf already processed 
File 127b.bdf already processed 
File 120a.bdf already processed 
File 126a.bdf already processed 
File 128b.bdf already processed 
File 130a.bdf already processed 
File 128a.bdf already processed 
File 129a.bdf already processed 
File 129b.bdf already processed 
File 130b.bdf already processed 
Processing file: 132a.bdf  
Not all events i

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# OLD:

#### Analyse mismatch response 

Deviant minus standard ERP
+ Check between subjects to see if the subjects have similar responses
+ Check between channels to observe which parts of the brain are more influenced by the events


#### Extract features 
+ peak latency
+ peak amplitude
+ mean amplitude


#### Create labels


#### Input data into DL models

#### Visualise results